## Load data

In [755]:
%%time
if False:
    # Load destinations
    import pandas as pd
    destinations = pd.read_csv('destinations.csv')
    destinations.head()

Wall time: 0 ns


In [299]:
%%time
#load test set
test_set = pd.read_csv('test.csv')
print('test data size:', test_set.shape)
#test_set.head

test data size: (2528243, 22)
Wall time: 18 s


In [756]:
%%time
if False:
    # subset with 10k user_id -- the train_sel_10k was created in the directory earlier. No need to load the whole train
    sel_train = pd.read_csv('sel_train_10k.csv')
    
    
if True:
    # load train set, the dataset is very large, we will work with a subset: (37670293, 24)
    # take 5 min to load
    train = pd.read_csv('train.csv')
    print('train data size', train.shape)
    #train.head

train data size (37670293, 24)
Wall time: 4min 55s


In [757]:
#train.info()
#test.shape
#test.info()
#train.head(5)
#test.head(5)

## What are we predicting? Given search, predict the hotel cluter booked!

## How are we scored? Mean Average Precision @ 5

In [758]:
if False:
    # distribution of hotel cluster
    %matplotlib inline
    sel_train['hotel_cluster'].hist(bins = 100)
    # distribution of count for each hotel cluster:
    #sel_train['hotel_cluster'].value_counts().hist(bins = 20)

## test user id is a subet of train's

In [759]:
'''
test_ids = set(test['user_id'])
train_ids = set(train['user_id'])
intersection_count = len(test_ids & train_ids)
print('test&train common unique ids:',intersection_count, 'test unique ids', len(test_ids))
'''

"\ntest_ids = set(test['user_id'])\ntrain_ids = set(train['user_id'])\nintersection_count = len(test_ids & train_ids)\nprint('test&train common unique ids:',intersection_count, 'test unique ids', len(test_ids))\n"

## Pick 10000 users with random user ids

In [760]:
if False:
    # when the full train is loaded, pick the subset and write it to the directory
    import random
    sel_user_id = random.sample(train_ids, 10000)
    sel_train = train[train['user_id'].isin(sel_user_id)]
    sel_train.to_csv('sel_train_10k.csv', index = False)

## Convert date

In [761]:
if False:
    sel_train["date_time"] = pd.to_datetime(sel_train["date_time"])
    #sel_train["year"] = sel_train["date_time"].dt.year
    #sel_train["month"] = sel_train["date_time"].dt.month

## distribution of counts of each user_id

In [762]:
#sel_train.groupby(sel_train['user_id'])['user_id'].count().hist(range = [0, 100], bins = 100)

In [763]:
#sel_train.groupby('user_id')['site_name'].count().reset_index(name = 'count').sort_values(['count'], ascending=False)
#sel_train['user_id'].value_counts()
# equivalent to value_counts for series

## distribution of counts of each day/month, Seperate to new training and testing set based on date

In [764]:
# set the time to be the index, this ease the date split
if False:
    
    sel_train = sel_train.set_index('date_time')
    sel_train['site_name'].resample('D').count().plot()
    sel_train['site_name'].resample('M').count().plot()

    ## Seperate to new training and testing set based on date
    sel_train
    len(sel_train['user_id'].unique())
    t1 = sel_train[sel_train.index < '2014-08-01']
    t2 = sel_train[sel_train.index >= '2014-08-01'] 

## Remove click events

In [765]:
if False:
    # when t2 is not the kaggle set,  t2['is_booking'] exists
    t2_b = t2[t2['is_booking'] == 1]
    t2_b.shape

# set variables of trainset and testset, can use local or Kaggle's Testset

In [766]:
if True:
    #t1 = sel_train
    t1 = train
    t2 = test_set

## Simple algorithm - predict as top 5 common cluster for all

In [812]:
%%time
if False:
    most_common_clusters = list(sel_train.hotel_cluster.value_counts().head(5).index)

if True:
    most_common_clusters = list(t1.hotel_cluster.value_counts().head(5).index)

Wall time: 526 ms


In [768]:
if False:
    # when t2 has the ground truth label
    predictions = [most_common_clusters for _ in range(t2.shape[0])]
    target = [[l] for l in t2['hotel_cluster']]

    import ml_metrics as metrics
    metrics.mapk(target, predictions, k=5)
    # check how mapk is calculated
    #[(apk(a,p,5),a,p) for a,p in zip(target, predictions)]

## Finding Correlations

In [769]:
# no column correlates to hotel cluster
#t1.corr()["hotel_cluster"]

# Create features (for machine learning)

In [770]:
if False:
    destinations.head()
    from sklearn.decomposition import PCA
    # only get the top 3 PCs
    pca = PCA(n_components =3)
    # transform all features to 3 PCs, call dest_small
    dest_small = pca.fit_transform(destinations[['d{0}'.format(i+1) for i in range(0, 149)]])
    dest_small = pd.DataFrame(dest_small)
    dest_small['srch_destination_id'] = destinations['srch_destination_id']
    
    ## Feature based o： train set + 3 Principal components from destinations
    # create date column based on the date index
    t1.loc[:, 'date_time'] = t1.index
    
    def calc_fast_features(t1):

        # convert to date_time format
        t1["date_time"] = pd.to_datetime(t1["date_time"])
        t1["srch_ci"] = pd.to_datetime(t1["srch_ci"], format='%Y-%m-%d', errors="coerce")
        t1["srch_co"] = pd.to_datetime(t1["srch_co"], format='%Y-%m-%d', errors="coerce")

        # quarter, month, day of week of the search log timestamp
        props = {}
        for prop in ["quarter", "month", "dayofweek", "day", "hour", "minute"]:
            props[prop] = getattr(t1['date_time'].dt, prop)

        # rest of the columns other than date_time, srch_ci/srch_co
        carryover = [p for p in t1.columns if p not in ["date_time", "srch_ci", "srch_co"]]
        for prop in carryover:
            props[prop] = t1[prop]

        # quarter, month, dayofweek, and day of chech in/check out date     
        date_props = ["month", "day", "dayofweek", "quarter"]
        for prop in date_props:
            props["ci_{}".format(prop)] = getattr(t1["srch_ci"].dt, prop)
            props["co_{}".format(prop)] = getattr(t1["srch_co"].dt, prop)

        # length of stay
        props["stay_span"] = (t1["srch_co"] - t1["srch_ci"]).astype('timedelta64[h]')

        ret = pd.DataFrame(props)

        # merge the destination feature

        ret = ret.join(dest_small, on = "srch_destination_id", how = 'left', rsuffix = "_dest")
        ret = ret.drop('srch_destination_id_dest', axis =1)
        return ret

    df = calc_fast_features(t1)
    df.fillna(-1, inplace=True)


## Machine learning attempt (show how a typical ML works)
### 1. This is predicting hotel cluster given a search event, no matter a booking or click event.
### 2. Simple random forest only predict 1 cluster, using accuracy for score, not map

In [771]:
# simple random forest with all features created

if False:
    df_rf = df[df["is_booking"] == 1]
    predictors = [c for c in df.columns if c not in ["hotel_cluster"]]
    from sklearn.model_selection import cross_val_score
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(n_estimators=10, max_depth = 50, min_weight_fraction_leaf=0.1)
    # this score is using mean accuracy
    scores = cross_val_score(clf, df_rf[predictors], df_rf['hotel_cluster'], cv=2)
    print(scores)

In [772]:
# confirm what the cross_val_score is doing, test KFold 
if False:
    # split the data into train and test, I only get the indices, not actual split
    from sklearn.model_selection import KFold
    KFold(2).split(df)
    for train, test in KFold(2).split(df):
        print(train, test)
    # train model and make predictions 
    clf.fit(df[predictors][98986:], df['hotel_cluster'][98986:])
    predictions_raw = clf.predict(df[predictors][:98986])

    ## test results
    # ML is doing somework, this is better than the baseline result
    target = [[l] for l in df["hotel_cluster"][:98986]]
    predictions = [[p] for p in predictions_raw]
    mapk(target, predictions, k=5)
    
    # no matter booking or no booking
    df['is_booking'].groupby(df['is_booking']).count()

In [773]:
## score of predicting a random cluster, a naive baseline
if False:
    # mapk of predicting a random cluster
    unique_clusters = set(df['hotel_cluster'].unique())
    import random
    predictions_random = [random.sample(unique_clusters,1) for _ in range(t2.shape[0])]
    mapk(target, predictions_random, 5)

## Binary classifiers 

In [774]:
if False:
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import KFold
    from itertools import chain
    
    #Build Binary Classifier for each hotel cluster
    #1. Loop through each hotel cluster.
    #  1. Create the new binary "label" for each row, indicates whether the row is in the cluster, or not.
    #  2. Train binary classifier for each of the K-fold and make predictions, extract the probablities of the cluster is "booked"
    #  3. Append the probabilities for a cluster to the whole probability list
    #2. Convert the whole probability list to data frame, transpose, add the column headers
    #3. For each row, find the 5 largest probabilities, and assign the hotel_cluster value as predictions
    #4. Compute accuracy via mapk
    
    all_probs = []
    unique_clusters = df['hotel_cluster'].unique()
    #1.
    for cluster in unique_clusters:
        df['target'] = 0
        df['target'][df['hotel_cluster'] == cluster] = 1
        predictors = [col for col in df.columns if col not in ['hotel_cluster','target']]
        probs = []
        cv = KFold(n_splits = 2)
        clf = RandomForestClassifier(n_estimators=10, min_weight_fraction_leaf=0.1)

        for train_index, test_index in cv.split(df):
            clf.fit(df[predictors].iloc[train_index], df["target"].iloc[train_index])
            preds = clf.predict_proba(df[predictors].iloc[test_index])
            probs.append([p[1] for p in preds])
        
        full_probs = chain.from_iterable(probs)
        all_probs.append(list(full_probs))
    #2.
    prediction_frame = pd.DataFrame(all_probs).T
    prediction_frame.columns = unique_clusters
    #3.
    def find_top_5(row):
        return list(row.nlargest(5).index)
    preds = []

    for index, row in prediction_frame.iterrows():
        preds.append(find_top_5(row))
    #4.
    mapk([[l] for l in df["hotel_cluster"]], preds, k = 5)

# Top cluster for each srch_destination_id

In [775]:
%%time
def make_key(items):
    return "_".join([str(i) for i in items])

match_cols = ["srch_destination_id"]
cluster_cols = match_cols + ['hotel_cluster']
# group by a list of columns
groups = t1.groupby(cluster_cols)

top_clusters = {}
# this return the groupby indices as a tuple, and the grouped data
for name, group in groups:
    
    clicks = len(group.is_booking[group.is_booking == False])
    bookings = len(group.is_booking[group.is_booking == True])

    score = bookings + .15 * clicks

    clus_name = make_key(name[:len(match_cols)])
    # if the srch_dest_id is not in the dict, set key as the srch_dest_id, value as another dict
    if clus_name not in top_clusters:
        top_clusters[clus_name] = {}
    # now the srch_dest_id is in the dict, find the dict for the srch_dest_id, add key = cluster, value = score
    top_clusters[clus_name][name[-1]] = score

Wall time: 6min 54s


In [776]:
## transform the dictionary to find the top k clusters
import operator
cluster_dict = {}
for srch_dest in top_clusters:
    cluster_w_score = top_clusters[srch_dest]
    #rank_top_5 = sorted(cluster_w_score, key = o)
    top_5 = [l[0] for l in sorted(cluster_w_score.items(), key = operator.itemgetter(1), reverse = True)[:5]]
    cluster_dict[srch_dest] = top_5

In [777]:
# make predictions for t2: for each row in t2, use its srch_destination_id to find the top 5, append empty if not exist
preds = []
for srch_destination_id in t2['srch_destination_id']:
    if str(srch_destination_id) in cluster_dict:
        pred = cluster_dict[str(srch_destination_id)]
        preds.append(pred)
    else:
        preds.append([])

if False:
    # if the ground truth is known
    print(mapk([[l] for l in t2["hotel_cluster"]], preds, k=5))

# Leakage solution: finding the exact matching users with leak of orig_destination_distance

In [778]:
# why leakage solution works? The distance defines the cluter!
#pd.set_option('display.max_columns', 999)
#t1[t1['user_id'] == 15286]
#t1[['user_id','is_booking', 'user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance', 'hotel_cluster']]

In [779]:
%%time
match_cols = ['user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance']

groups = t1.groupby(match_cols)

def generate_exact_matches(row, match_cols):
    index = tuple([row[t] for t in match_cols])
    # see if the t2 cols can be found in t1
    try:
        group = groups.get_group(index)
    except Exception:
        return []
    # if found, return hotel_cluster(s)
    clus = list(set(group.hotel_cluster))
    return clus

exact_matches = []
for i in range(t2.shape[0]):
    exact_matches.append(generate_exact_matches(t2.iloc[i], match_cols))

Wall time: 16min 34s


In [780]:
exact_matches[0] + preds[0] + most_common_clusters 

[5, 37, 55, 11, 22, 91, 41, 48, 64, 65]

# Combine 3 solutions

In [781]:
def f5(seq, idfun=None):
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result

full_preds = [f5(exact_matches[p] + preds[p] + most_common_clusters)[:5] for p in range(len(preds))]

if False:
    # work when t2['hotel_cluster'] exists
    mapk([[l] for l in t2['hotel_cluster']], full_preds, k = 5)
import os
os.getcwd()

'C:\\Users\\Di Fu\\Desktop\\Di\\Expedia'

# Write result to a CSV

In [808]:
%%time
write_p = [" ".join([str(l) for l in p]) for p in full_preds]
write_frame = ["{0},{1}".format(test_set["id"][i], write_p[i]) for i in range(len(full_preds))]
write_frame = ["id,hotel_cluster"] + write_frame
with open("predictions.csv", "w+") as f:
    f.write("\n".join(write_frame))

Wall time: 1min 5s


In [809]:
upload_set = pd.read_csv('predictions.csv')


In [810]:
len(full_preds)

2528243

In [811]:
len(upload_set)

2528243